# E-Commerce Data EDA and Cleaning
Our dataset contains thousands of online purchases. 

In [1]:
# importing our libraries
import pandas as pd

In [2]:
# reading our data
data = pd.read_csv('data/data.csv')

# Exploratory Data Analysis

In [3]:
# Let's take a look at our data.
print(data.shape) # How big is our dataset?
data.head() # What do the first few rows look like?

(541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [4]:
# What data types are we working with?
print(data.dtypes)

InvoiceNo       object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
UnitPrice      float64
CustomerID     float64
Country         object
dtype: object


The dataset includes invoice numbers (unique identifiers for each invoice), stock codes (unique identifiers for each product), product descriptions, product quantity, unit price, customer IDs (unique identifiers for each customer), and country.  

A few observations:
- Each row only includes a specific product from a specific order. This means that there could be multiple rows for a single invoice number is a customer purchased different products in the same order. For exampke, it appears that our first five rows all have the same 
- Each stock code should have a single unit price.

It looks like almost everything is in order. Quantity is an integer, as it should be; Unit Price is a float, which is correct.  
Our UIDs (InvoiceNo, StockCode, and CustomerID) could be integers or objects - it doesn't make a huge difference. However, because unique identifiers could theoretically contain letters and there would be no reason to perform mathematical calculations with them, I will make them objects.  
I will also change the InvoiceDate column to a datetime object.
## To-Dos
- Change CustomerID type to string.
- Change InvoiceDate type to datetime.

In [5]:
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate']) # changes InvoiceDate data type to datetime.
data['CustomerID'] = data['CustomerID'].astype(str)

In [6]:
# Let's take a look at our numeric columns.
data.describe()

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


# Cleaning
Looking at our numeric columns reveals a few issues. First, the minimums are both negative, even though we know you can't order a negative quantity of items for a negative price online. Also, the maximums for Quantity and UnitPrice are both obscenely high. It is not impossible that someone could have ordered 80,000 of an item or an item that costs almost \$40,000 but it seems unlikely.  
We found earlier that we have over half a million rows in this dataset so it is likely safe to remove rows with negative Quantity or UnitPrice values.  
What should we do with the possible but implausible outliers? Since we have a Description column, let's take a look at a couple outlier rows to see if they seem plausible.  
## To-Dos
- Check outliers.

In [7]:
# Before we continue, let's check how much of our data is missing.
data.isnull().sum()

InvoiceNo         0
StockCode         0
Description    1454
Quantity          0
InvoiceDate       0
UnitPrice         0
CustomerID        0
Country           0
dtype: int64

Apparently only 1454 rows have missing data, and all the missing data is from the Description column. This missing data is tolerable - the Description column is not essential so we do not need to remove these rows just become of missing Descpription data. Let's keep exploring.

In [8]:
# Let's say outliers of the Quantity colum have quantities of at least 10,000.
quantity_outliers = data.loc[data['Quantity'] >= 10000]
# And UnitPrice outliers have prices of at least $10,000.
price_outliers = data.loc[data['UnitPrice'] >= 10000]
# How many outliers do we get?
print(quantity_outliers.shape)
print(price_outliers.shape)

(3, 8)
(10, 8)


In [9]:
# What do our Quantity outliers look like?
quantity_outliers.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom
502122,578841,84826,ASSTD DESIGN 3D PAPER STICKERS,12540,2011-11-25 15:57:00,0.00,13256.0,United Kingdom
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom


In [10]:
# What do our UnitPrice outliers look like?
price_outliers.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
15016,C537630,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:04:00,13541.33,nan,United Kingdom
15017,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,nan,United Kingdom
16232,C537644,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:34:00,13474.79,nan,United Kingdom
16356,C537651,AMAZONFEE,AMAZON FEE,-1,2010-12-07 15:49:00,13541.33,nan,United Kingdom
43702,C540117,AMAZONFEE,AMAZON FEE,-1,2011-01-05 09:55:00,16888.02,nan,United Kingdom
43703,C540118,AMAZONFEE,AMAZON FEE,-1,2011-01-05 09:57:00,16453.71,nan,United Kingdom
222681,C556445,M,Manual,-1,2011-06-10 15:31:00,38970.00,15098.0,United Kingdom
299982,A563185,B,Adjust bad debt,1,2011-08-12 14:50:00,11062.06,nan,United Kingdom
524601,C580604,AMAZONFEE,AMAZON FEE,-1,2011-12-05 11:35:00,11586.50,nan,United Kingdom
524602,C580605,AMAZONFEE,AMAZON FEE,-1,2011-12-05 11:36:00,17836.46,nan,United Kingdom


## Outliers
The Quantity outliers are weird. Somebody apparently spent about \\$75,000 on ceramic jars, and someone else spent over \\$160,000 on little paper birdies. And someone else purchases 12,540 stickers, but seemingly at no charge. What do we make of this? Do we even want to consider items with a price of \$0? Maybe, maybe not - on one hand, those purchases provide no value to us. On the other, those customers may spend plenty on other products. I would argue that we should remove data where the UnitPrice is 0 because these transactions likely hold little value for us, assuming we make money by selling products or running the website.  
What about the UnitPrice outliers? Almost all the price outliers are weird Amazon fees. Only one of them had a CustomerID. The rest are nan's.  
## Unexplained Phenomena
Wait, we already checked for null values, and CustomerID supposedly had none! What's going on? Does CustomerID have a bunch of string nan's? This requires further investigation.
## To-Dos
- Remove rows with Quantity < 0.
- Remove rows with UnitPrice <= 0.
- Remove rows with outliers (Quantity > 10,000, UnitPrice > 10,000).
- Check if CustomerID sometimes equals 'nan'. If so, remove those rows.

In [11]:
# Let's remove rows where Quantity < 0
data = data.loc[data['Quantity'] >= 0]

# Let's remove rows where UnitPrice <= 0.
data = data.loc[data['UnitPrice'] > 0]

# Let's remove our outliers.
data = data.loc[data['Quantity'] < 10000]
data = data.loc[data['UnitPrice'] < 10000]

# Finally, let's figure out what's going on with the nan's.
nanCustomerID = data.loc[data['CustomerID'] == 'nan']

In [12]:
print(nanCustomerID.shape)
nanCustomerID.head()

(132218, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,2010-12-01 14:32:00,2.51,nan,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,2010-12-01 14:32:00,2.51,nan,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,2010-12-01 14:32:00,0.85,nan,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,2010-12-01 14:32:00,1.66,nan,United Kingdom
1447,536544,21790,VINTAGE SNAP CARDS,9,2010-12-01 14:32:00,1.66,nan,United Kingdom


Wow - 132,218 rows have 'nan' as the CustomerID. That's almost a quarter of the data. It's not the end of the world since we have so much data, and we would still have over 400,000 rows remaining if we removed all the rows with 'nan' as the Customer ID. What do we do?  
Our next step really depends on the type of analysis we are interested in. The CustomerID column tells us if people are making multiple purchases. Without it, we can't determine if certain people tend to buy certain products, or make purchases of certain sizes at certain times. There are many analyses we cannot perform on data with missing data in the CustomerID column.  
If we do decide that we are going to remove this data, we need to make sure that the data in the 'nan' CustomerID subset is not different from the rest of the dataset. We want to make sure that removing the 'nan' CustomerID subset will not impact the remaining dataset. For example, maybe 'nan' in the CustomerID means the customer used a Guest login, and maybe people who use Guest logins are more likely to make small purchases. Let's check.

## To Do
- Check if the 'nan' CustomerID subset is similar to the remaining dataset. If so, we can remove the 'nan' CustomerID subset.

In [13]:
# Let's get all rows where CustomerID does not equal 'nan'.
remainder = data.loc[data['CustomerID'] != 'nan']

remainder.describe()

,Quantity,UnitPrice
count,397882.000000,397882.000000
mean,12.598213,3.116496
std,42.886003,22.097932
min,1.000000,0.001000
25%,2.000000,1.250000
50%,6.000000,1.950000
75%,12.000000,3.750000
max,4800.000000,8142.750000


In [14]:
nanCustomerID.describe()

,Quantity,UnitPrice
count,132218.000000,132218.000000
mean,3.180823,6.102375
std,11.199243,37.211048
min,1.000000,0.060000
25%,1.000000,1.630000
50%,1.000000,3.290000
75%,3.000000,5.790000
max,1820.000000,4505.170000


## Can we remove the rows missing CustomerID?
Unfortunately, no. The quantity and unit price data from the CustomerID-less data have lower means and medians than the rest of the data, suggesting that removing the CustomerID-less purchase data would affect the dataset overall. We do have a couple of courses of action, though:
- Some analyses may simply not need CustomerID. In these cases, we're good to go.
- What we do have are invoice numbers. These invoice numbers link individual rows from the dataset (each row represents a quantity of a single item) that were part of the same order. We could treat CustomerID-less purchases with the same invoice number as having been made by a new user. This would mean that for the CustomerID-less data, each unique invoice number would be assigned a unique user.  

The latter course of action makes an almost certainly incorrect assumption: that each CustomerID-less order was purchased by a unique customer. It is likely that certain customers made multiple purchases without a CustomerID. However, because we have no way of identifying these customers, treating each invoice as a new customer is our best course of action.

## To-Dos
- Replace 'nan' CustomerIDs with automatically generated CustomerIDs. These CustomerIDs will be 'U' + InvoiceNo. For example, if the InvoiceNo '12345' has a 'nan' CustomerID, the generated CustomerID would be 'U12345'. The 'U' prefix signifies that the CustomerID is a place-holder.

In [15]:
# For each row with a 'nan' CustomerID, assign it a new 'U' + InvoiceNo ID.
data.loc[(data['CustomerID'] == 'nan'), 'CustomerID'] = 'U' + data['InvoiceNo']

In [16]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## And we're good to go!
Our data has been cleaned and we're ready to run some analyses on it. If we ever decide to exclude CustomerID-less purchases, we can easily filter them out because of their 'U' prefix. Let's save our data and get to the fun stuff.

In [17]:
data.to_csv('data/cleaned_data.csv', index=False)